In [58]:
import pandas as pd 
import os
import numpy as np
import torch
from skimage.io import imread
from sklearn import svm
from skimage.feature import hog
from skimage.segmentation import slic
import torchvision

In [3]:
df = pd.read_csv('/home/giorgia/Desktop/MAI/Thesis/patch_info.csv')
len(df[df.plastic!=-1])

1161

In [4]:
df = pd.read_csv('/home/giorgia/Desktop/MAI/Thesis/patch_info.csv')
df = df[df.plastic!=-1]
df_plastic = df[df.plastic==1].reset_index()
df_noplastic = df[df.plastic==0].reset_index()
len(df_plastic), len(df_noplastic)


(567, 594)

In [5]:
partition = 0.89

rand_ind_p = np.random.randint(0, len(df_plastic), int(partition*len(df_plastic)))
rand_ind_np = np.random.randint(0, len(df_noplastic), int(partition*len(df_noplastic)))

training_set = pd.concat([df_plastic.loc[rand_ind_p, ['patch_filename', 'plastic']], df_noplastic.loc[rand_ind_np, ['patch_filename', 'plastic']]]).reset_index()
validation_set = pd.merge(df.loc[:, ['patch_filename', 'plastic']], training_set, indicator=True, how='outer').query('_merge=="left_only"').drop(['_merge','index'], axis=1).reset_index()

len(training_set), len(validation_set)

(1032, 480)

In [71]:
def prepareData(df):
    data = []
    labels = []
    patches_path = '/home/giorgia/Desktop/MAI/Thesis/images/patches/'
    for i in range(len(df)):
        patch_filename = os.path.join(patches_path, os.path.basename(df.loc[i, 'patch_filename']))
        #im = np.amax(imread(patch_filename), axis=2) # maximum intensity projection
        im = imread(patch_filename)
        if im.shape == (256, 256, 3):
            #feat = hog(im, feature_vector=True)
            #feat, _ = np.histogram(im, bins = 100)
            feat = slic(im).flatten()
            #feat_descriptor.detect_and_extract(im)
            #feat = feat_descriptor.descriptors
            data.append(feat)
            #data.append(im.flatten()) # use mip intensities as feature vectors
            labels.append(df.loc[i, 'plastic'])
    return data, labels


In [72]:
training_data, training_labels = prepareData(training_set)
test_data, test_labels = prepareData(validation_set)

In [73]:
len(training_data[2])

65536

In [74]:
rbf_svc = svm.SVC(kernel='rbf')
rbf_svc.fit(training_data, training_labels)
rbf_svc.score(test_data, test_labels)

In [56]:
from sklearn.ensemble import RandomForestClassifier 

rfc = RandomForestClassifier()
rfc.fit(training_data, training_labels)
rfc.score(test_data, test_labels)


0.7948717948717948

In [6]:
def loadBatch(img_df, batch_size=16):
    if len(img_df)<=batch_size:
        X = [imread(img_df.loc[i,'patch_filename']) for i in img_df.index.values.to_list()]
        Y = img_df.plastic.values.to_list()
    else:
        shuffled_ind = np.random.randi(0, img_df.index.max(), batch_size)
        X = [imread(img_df.loc[i,'patch_filename']) for i in shuffled_ind]
        Y = img_df.loc[shuffled_ind,'plastic'].values
        img_df = img_df.drop(img_df.index[shuffled_ind])
    return X, Y, img_df

In [7]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
dataset = datasets.ImageFolder('path', transform=transform)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')